# FACE AND EYE RECOGNITION USING VIOLA&JONES (HAAR CASCADE CLASSIFIER)

This notebook includes a code for face and eye recognition from our webcam images using Haar Cascade classifier. Once eyes are extracted from the original image, that images is cropped and used to predict, using our model trainned with our dataset, if the eyes are opened or closed.

Haar Cascade have some issues detecting closed eyes as eyes, also it can detect noise holes, chin or mouth as eyes and our model also have some problems classifing this closed eyes as closed, but its an initial code.

Object Detection using Haar feature-based cascade classifiers is an effective object detection method proposed by Paul Viola and Michael Jones in their paper, "Rapid Object Detection using a Boosted Cascade of Simple Features" in 2001. It is a machine learning based approach where a cascade function is trained from a lot of positive and negative images. It is then used to detect objects in other images.

Initially, the algorithm needs a lot of positive images (images of faces) and negative images (images without faces) to train the classifier. Then we need to extract features from it. For this, Haar features shown in the below image are used. They are just like our convolutional kernel. Each feature is a single value obtained by subtracting sum of pixels under the white rectangle from sum of pixels under the black rectangle.

<img src="notebook_images/haar.jpeg">

OpenCV library includes some models already trained for many object detections, for our case there are models for face and eye detection, that can be easily used with Python.

This is a old model that gives quite good results and can be used at real-time applications, but Haar cascades are notoriously prone to false-positives — the Viola-Jones algorithm can easily report a face in an image when no face is present.

More modern and accurate models must be taken into account, as this method has now been far surpassed by other methods, such as using Histogram of Oriented Gradients (HOG) + Linear SVM and deep learning (CNN, YOLO). 

In [1]:
import numpy as np
import cv2
from keras.models import load_model
from skimage.transform import resize
import os
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
def load_haarcascade():
    face_cascade_name = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    eye_cascade_name = cv2.data.haarcascades + 'haarcascade_eye.xml'
    
    face_cascade = cv2.CascadeClassifier()
    if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
        print("Error loading xml file")
        exit(0)

    eye_cascade = cv2.CascadeClassifier()
    if not eye_cascade.load(cv2.samples.findFile(eye_cascade_name)):
        print("Error loading xml file")
        exit(0)
        
    return face_cascade, eye_cascade

In [3]:
#LOAD HAAR CASCADE MODELS TO DETECT FACE AND EYES
face_cascade, eye_cascade = load_haarcascade()

#LAUNCH VIDEOCAMARA
cap = cv2.VideoCapture(0)

#LOAD TRAINNED MODEL FOR EYE CLASSIFIER
model = load_model('model/eye_classifier1.h5')

fontScale = 1
font = cv2.FONT_HERSHEY_PLAIN
thickness = 2

while 1:

    #READING IMAGE FROM VIDEOCAMERA
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #DETECT FACES FROM IMAGE
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    #LOOP FOR FACES
    for (x,y,w,h) in faces:
        
        #DRAW RECTANGLE OVER FACES
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        
        #CROPPED IMAGE OF ONLY FACE
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        
        #DETECT EYES ON FACE IMAGE
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        for (ex,ey,ew,eh) in eyes:
            
            #CROPPED IMAGE OF ONLY EYES
            eye_color = roi_color[ey:ey+eh, ex:ex+ew]
            
            #RESIZE IMAGE FOR NN EYE CLASSIFIER (OPENED OR CLOSED)
            eye_color_scaled = resize(eye_color, (200, 200), preserve_range=True).astype(np.uint8)
            
            #DRAW RECTANGLE OVER EYES
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            
            #PREDICT IF EYE IS OPENED OR CLOSED
            out_probabilities = model.predict(np.reshape(eye_color_scaled,(1,200,200,3)))
            result = "OPENED" if out_probabilities[0][0] > 0.5 else "CLOSED"
            text_x = int(ex+ew/4)
            text_y = int(ey-20)
            
            #DRAW TEXT OVER EYES
            cv2.putText(roi_color, result, (text_x, text_y), font, fontScale, (255,0,0), thickness, cv2.LINE_AA)
            
    cv2.imshow('img',img)
    #PRESS ESC TO STOP PROGRAM
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()